# ORA
the general and specfic for KEGG, REACTOME and GO

In [1]:
import sys
sys.path.insert(0, "/Users/yangxu/PyPathway")

In [2]:
import pandas as pd
from pypathway import Reactome, GO, KEGG, ORA
from pypathway import ColorectalCancer, IdMapping, GMTUtils
from pypathway import EnrichmentExport
import os

In [3]:
# load a gmt file.
gmt = GMTUtils.parse_gmt_file("../../pypathway/tests/gmt_file/h.all.v6.0.entrez.gmt")

In [4]:
# load the example
c = ColorectalCancer()

In [5]:
# infomation of datasets
len(c.deg_list), len(c.background)

(5320, 17216)

In [6]:
res_h = ORA.run(c.deg_list, c.background, gmt)

In [7]:
res_h.table.head()

,name,mapped,number in study,p-value,fdr
0,HALLMARK_MITOTIC_SPINDLE,182,77,7.125914e-04,1.370368e-03
1,HALLMARK_ESTROGEN_RESPONSE_LATE,191,100,4.735396e-10,2.630776e-09
2,HALLMARK_UV_RESPONSE_UP,152,63,3.682014e-03,5.753147e-03
3,HALLMARK_INFLAMMATORY_RESPONSE,184,68,4.550977e-02,6.692613e-02
4,HALLMARK_GLYCOLYSIS,184,91,9.956407e-08,3.555860e-07


In [8]:
# inline bar plot.
res_h.plot()

## KEGG

In [9]:
# kegg enrichment for certain organism
r_kg = KEGG.run(c.deg_list, c.background, 'hsa')

In [10]:
r_kg.table.head()

,ID,Name,mapped,deg,p-value,fdr
0,hsa00310,Lysine degradation - Homo sapiens (human),53,22,0.066158,0.154538
1,hsa04810,Regulation of actin cytoskeleton - Homo sapien...,188,84,0.000045,0.000628
2,hsa00051,Fructose and mannose metabolism - Homo sapiens...,31,14,0.066710,0.154538
3,hsa00220,Arginine biosynthesis - Homo sapiens (human),18,5,0.697038,0.815440
4,hsa04151,PI3K-Akt signaling pathway - Homo sapiens (human),311,111,0.038540,0.105415


In [11]:
r_kg.plot()

## Reactome

In [12]:
# the Example of using the warpper of Reactome gene set enrichment analysis

In [7]:
sybs = [x[1][0] for x in IdMapping.convert(input_id=c.deg_list, species='hsa', source='ENTREZID', target='SYMBOL') if x[1]]

In [8]:
sybs[:10]

['A2M',
 'MKKS',
 'S100A3',
 'ANKRD29',
 'TMEM250',
 'NAT1',
 'NAT2',
 'SERPINA3',
 'AAMP',
 'AARS']

In [9]:
# the input is a list of symbol
r = Reactome.run(sybs[:10], organism='Homo sapiens')

In [10]:
# the result
r.table.head()

,name,dbId,found,p-value,fdr,species
0,Acetylation,156582,2,0.000557,0.023960,Homo sapiens
1,Defective SLC6A2 causes orthostatic intoleranc...,5619109,1,0.006418,0.131517,Homo sapiens
2,Amino acid transport across the plasma membrane,352230,2,0.009394,0.131517,Homo sapiens
3,Neurotransmitter uptake and metabolism In glia...,112313,1,0.021240,0.162962,Homo sapiens
4,Astrocytic Glutamate-Glutamine Uptake And Meta...,210455,1,0.021240,0.162962,Homo sapiens


## Gene ontology

In [18]:
# make the association file using the id_mapping function
# detail using will be shown in the utils section
r = IdMapping.convert_to_dict(input_id=c.background, source='ENTREZID', target="GO", organism='hsa')

In [19]:
# run go enrichment analysis via goatools 
# the inputs of study, pop, and assoc is list, list,  dict.
# the path is the folder of go obo file
# the path should be a valid filesystem path
path = os.getcwd() + "/go.obo"
rg = GO.run([str(x) for x in c.deg_list], [str(x) for x in c.background], r, obo=path)

load obo file /Users/yangxu/PyPathway/examples/analysis/go.obo
/Users/yangxu/PyPathway/examples/analysis/go.obo: fmt(1.2) rel(2017-04-28) 48,589 GO Terms
fisher module not installed.  Falling back on scipy.stats.fisher_exact


Propagating term counts to parents ..
goids not found: {'GO:0140069', 'GO:0102752', 'GO:0103066', 'GO:0102113', 'GO:0106046', 'GO:0102389', 'GO:0140052', 'GO:0102486', 'GO:0120092', 'GO:0120103', 'GO:0102158', 'GO:0102966', 'GO:0140080', 'GO:0061896', 'GO:0102491', 'GO:0120094', 'GO:0061890', 'GO:0140059', 'GO:0102499', 'GO:0140078', 'GO:0140065', 'GO:0102250', 'GO:0140067', 'GO:0120058', 'GO:0102485', 'GO:0061931', 'GO:0120072', 'GO:0102102', 'GO:0140031', 'GO:0061899', 'GO:0102500', 'GO:0140068', 'GO:0102200', 'GO:0102524', 'GO:0120108', 'GO:0102756', 'GO:0120053', 'GO:0061921', 'GO:0102567', 'GO:0110053', 'GO:0102769', 'GO:0106030', 'GO:0102797', 'GO:0102488', 'GO:0102420', 'GO:0102751', 'GO:0102148', 'GO:0103067', 'GO:0102487', 'GO:0102568', 'GO:0102707', 'GO:0106028', 'GO:0140036', 'GO:0061920', 'GO:0140066', 'GO:0102489', 'GO:0106044', 'GO:0061885', 'GO:0102294', 'GO:0140058', 'GO:0061897', 'GO:0110026', 'GO:0106022', 'GO:0102953', 'GO:0103045', 'GO:0102773', 'GO:0102798', 'GO:01

15,292 out of 17,216 population items found in association
Calculating uncorrected p-values using fisher_scipy_stats
 5,114 out of  5,320 study items found in association
Running multitest correction: local bonferroni
Running multitest correction: local sidak
Running multitest correction: local holm
  15,947 GO terms are associated with 5,114 of 5,320 study items
  21,008 GO terms are associated with 15,292 of 17,216 population items


In [22]:
# the file input of study, pop and assoc
# this function is the warpper of the Goatools
# Github: https://github.com/tanghaibao/goatools
# cite: Haibao Tang et al. (2015). GOATOOLS: Tools for Gene Ontology. Zenodo. 10.5281/zenodo.31628.

path = "/Users/yangxu/goatools/"
rg = GO.run(path + 'data/study', path + 'data/population', path + 'data/association',
        obo=path + 'go-basic.obo')

load obo file /Users/yangxu/goatools/go-basic.obo
/Users/yangxu/goatools/go-basic.obo: fmt(1.2) rel(2017-04-28) 48,589 GO Terms
fisher module not installed.  Falling back on scipy.stats.fisher_exact


Propagating term counts to parents ..
goids not found: {'GO:0022625', 'GO:0022627'}


31,855 out of 33,239 population items found in association
Calculating uncorrected p-values using fisher_scipy_stats
   269 out of    276 study items found in association
Running multitest correction: local bonferroni
Running multitest correction: local sidak
Running multitest correction: local holm
  797 GO terms are associated with 269 of 276 study items
  6,122 GO terms are associated with 31,855 of 33,239 population items


In [23]:
rg.table.head()

,GO,NS,enrichment,name,ratio_in_study,ratio_in_pop,p_uncorrected,depth,study_count,p_bonferroni,p_sidak,p_holm,hit
0,GO:0006464,BP,e,cellular protein modification process,33/276,1727/33239,0.000008,n.a.,33,0.0505,0.0492,0.0505,"AT1G13580, AT1G66610, AT1G66860, AT1G66980, AT..."
1,GO:0036211,BP,e,protein modification process,33/276,1727/33239,0.000008,n.a.,33,0.0505,0.0492,0.0505,"AT1G13580, AT1G66610, AT1G66860, AT1G66980, AT..."
2,GO:0006468,BP,e,protein phosphorylation,22/276,922/33239,0.000011,n.a.,22,0.0660,0.0643,0.0660,"AT1G66980, AT2G29220, AT2G41140, AT2G41970, AT..."
3,GO:0016310,BP,e,phosphorylation,22/276,996/33239,0.000035,n.a.,22,0.2130,0.2070,0.2120,"AT1G66980, AT2G29220, AT2G41140, AT2G41970, AT..."
4,GO:0043412,BP,e,macromolecule modification,33/276,1877/33239,0.000057,n.a.,33,0.3510,0.3420,0.3510,"AT1G13580, AT1G66610, AT1G66860, AT1G66980, AT..."


In [24]:
# the interactive graph display the significance of the result by color
rg.graph()

In [25]:
# test export
c = EnrichmentExport.export([rg, rg])